In [2]:
import xarray as xr
import dask
from dask.distributed import Client
import matplotlib.pyplot as plt
import numpy as np

In [3]:
client = Client()
client

/home/dyllanjr/anaconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46435 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:46241 Dashboard: http://127.0.0.1:46435/status,Cluster Workers: 4 Cores: 12 Memory: 50.56 GB


In [10]:
#I'll need to group by year and then apply the following on each year dataset:
def make_features(ds):
    """Function to apply to every year of the 79-2020 dset
    INPUT
    ds: an xarray dataset, one year's worth
    
    OUTPUT
    dset: an xarray dataset, one year's worth of feature engineered data"""
    
    #Min monthly value for whole year
    annual_min = ds.min(dim='time')
    annual_min = annual_min.rename({'t2m':'annual_min_t2m', 'tp':'annual_min_tp'})
    
    #Max monthly value for whole year
    annual_max = ds.max(dim='time')
    annual_max = annual_max.rename({'t2m':'annual_max_t2m', 'tp':'annual_max_tp'})
    
    #Annual mean value
    annual_mean = ds.mean(dim='time')
    annual_mean = annual_mean.rename({'t2m':'annual_mean_t2m', 'tp':'annual_mean_tp'})
    
    north_hemi = ds.where(ds.latitude > 0, drop = True)
    south_hemi = ds.where(ds.latitude <= 0, drop =  True)

    north_hemi = ds.where(ds.latitude > 0, drop = True)
    south_hemi = ds.where(ds.latitude <= 0, drop =  True)

    def as_mask(month):
        #creates boolean mask for summer/winter months (april-sept), depending on hemisphere
        return (month >= 4) & (month <= 9)

    def om_mask(month):
        #creates boolean mask for summer/winter months (oct-march), depending on hemisphere
        return (month <= 3) | (month >= 10)
    
    #Min/max monthly value for summer months (april-sept) for Northern Hemi
    north_hemi_summer = north_hemi.sel(time=as_mask(north_hemi['time.month']))

    north_hemi_summer_min = north_hemi_summer.min(dim = 'time')
    north_hemi_summer_min = north_hemi_summer_min.rename({'t2m': 'sum_min_t2m', 'tp': 'sum_min_tp'})

    north_hemi_summer_max = north_hemi_summer.max(dim = 'time')
    north_hemi_summer_max = north_hemi_summer_max.rename({'t2m': 'sum_max_t2m', 'tp': 'sum_max_tp'})

    #Min/max monthly value for summer months (oct-march) for southern hemi
    south_hemi_summer = south_hemi.sel(time = om_mask(south_hemi['time.month']))

    south_hemi_summer_min = south_hemi_summer.min(dim = 'time')
    south_hemi_summer_min = south_hemi_summer_min.rename({'t2m': 'sum_min_t2m', 'tp': 'sum_min_tp'})

    south_hemi_summer_max = south_hemi_summer.max(dim = 'time')
    south_hemi_summer_max = south_hemi_summer_max.rename({'t2m': 'sum_max_t2m', 'tp': 'sum_max_tp'})

    #min/max monthly value for winter months (oct-march) for Northern hemi
    north_hemi_winter = north_hemi.sel(time=om_mask(north_hemi['time.month']))

    north_hemi_winter_min = north_hemi_winter.min(dim = 'time')
    north_hemi_winter_min = north_hemi_winter_min.rename({'t2m': 'win_min_t2m', 'tp': 'win_min_tp'})

    north_hemi_winter_max = north_hemi_winter.max(dim = 'time')
    north_hemi_winter_max = north_hemi_winter_max.rename({'t2m': 'win_max_t2m', 'tp': 'win_max_tp'})

    #min/max monthly value for winter months (april-sept) for southern hemi
    south_hemi_winter = south_hemi.sel(time = as_mask(south_hemi['time.month']))

    south_hemi_winter_min = south_hemi_winter.min(dim = 'time')
    south_hemi_winter_min = south_hemi_winter_min.rename({'t2m': 'win_min_t2m', 'tp': 'win_min_tp'})

    south_hemi_winter_max = south_hemi_winter.max(dim = 'time')
    south_hemi_winter_max = south_hemi_winter_max.rename({'t2m': 'win_max_t2m', 'tp': 'win_max_tp'})
    
    #mean summer months
    north_hemi_summer_mean = north_hemi_summer.mean(dim = 'time')
    north_hemi_summer_mean = north_hemi_summer_mean.rename({'t2m': 'sum_mean_t2m', 'tp': 'sum_mean_tp'})

    south_hemi_summer_mean = south_hemi_summer.mean(dim = 'time')
    south_hemi_summer_mean = south_hemi_summer_mean.rename({'t2m': 'sum_mean_t2m', 'tp': 'sum_mean_tp'})

    #mean winter months
    north_hemi_winter_mean = north_hemi_winter.mean(dim = 'time')
    north_hemi_winter_mean = north_hemi_winter_mean.rename({'t2m': 'win_mean_t2m', 'tp': 'win_mean_tp'})


    south_hemi_winter_mean = south_hemi_winter.mean(dim = 'time')
    south_hemi_winter_mean = south_hemi_winter_mean.rename({'t2m': 'win_mean_t2m', 'tp': 'win_mean_tp'})
    
    summer_min = xr.combine_nested([[north_hemi_summer_min],[south_hemi_summer_min]], concat_dim=['latitude', 'longitude'])
    summer_max = xr.combine_nested([[north_hemi_summer_max],[south_hemi_summer_max]], concat_dim=['latitude', 'longitude'])
    summer_mean = xr.combine_nested([[north_hemi_summer_mean],[south_hemi_summer_mean]], concat_dim=['latitude', 'longitude'])

    winter_min = xr.combine_nested([[north_hemi_winter_min],[south_hemi_winter_min]], concat_dim=['latitude', 'longitude'])
    winter_max = xr.combine_nested([[north_hemi_winter_max],[south_hemi_winter_max]], concat_dim=['latitude', 'longitude'])
    winter_mean = xr.combine_nested([[north_hemi_winter_mean],[south_hemi_winter_mean]], concat_dim=['latitude', 'longitude'])
    
    dset = xr.combine_by_coords([annual_min, annual_max, annual_mean, summer_min, summer_max, summer_mean, winter_min, winter_max, winter_mean])
    
    return dset

In [5]:
ds = xr.open_dataset('monthly_era5_79_20.nc',
                     chunks={'time': -1, 'latitude': 'auto', 'longitude': 'auto'})

In [6]:
ds

<xarray.Dataset>
Dimensions:    (expver: 2, latitude: 129, longitude: 173, time: 500)
Coordinates:
  * longitude  (longitude) float32 -18.5 -18.25 -18.0 -17.75 ... 24.0 24.25 24.5
  * latitude   (latitude) float32 23.5 23.25 23.0 22.75 ... -8.0 -8.25 -8.5
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2020-08-01
Data variables:
    t2m        (time, expver, latitude, longitude) float32 dask.array<chunksize=(500, 2, 129, 173), meta=np.ndarray>
    tp         (time, expver, latitude, longitude) float32 dask.array<chunksize=(500, 2, 129, 173), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-09-25 18:31:21 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [48]:
years = ds.groupby('time.year')
year_list = []
for ix, year in years:
    year_list.append(ix)
    if ix == 1979:
        dset = make_features(year)
    else:
        year_set = make_features(year)
        dset = xr.concat([dset, year_set], dim='time')
    
    

In [51]:
dset['time'] = year_list
dset['time']

<xarray.DataArray 'time' (time: 42)>
array([1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020])
Coordinates:
  * time     (time) int64 1979 1980 1981 1982 1983 ... 2016 2017 2018 2019 2020

In [52]:
dset

,Array,Chunk
Bytes,7.50 MB,178.54 kB
Shape,"(42, 2, 129, 173)","(1, 2, 129, 173)"
Count,1072 Tasks,42 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,7.50 MB,178.54 kB
Shape,"(42, 2, 129, 173)","(1, 2, 129, 173)"
Count,1072 Tasks,42 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [53]:
dset.to_netcdf('era5_79_20_prepared.nc')